# Adding Coordinates to the Datafrane using geocode from Google

### Calling the necessary dependencies

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import pprint

# Google API Key
from config import gkey

### Import the census data by County

In [4]:
coordinates_pd = pd.read_csv("../1_census/output/census_data_county.csv")
coordinates_pd

,State,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,Households with Internet,Households with Just Cell Data Plan,Households with DSL or Fiber Optic,Households with Broadband Any Type,Households with Celular Data Plan,Households with Just DSL or Fiber Optic,Households with Satellite Internet,Households with Just Satellite Internet,Households with No Internet
0,72,"Corozal Municipio, Puerto Rico",34933.0,38.1,14752.0,7887.0,19146.0,54.807775,9.435205,11026.0,2381.0,2268.0,5021.0,4088.0,706.0,361.0,161.0,5686.0
1,72,"Maunabo Municipio, Puerto Rico",11297.0,43.5,17636.0,8830.0,5945.0,52.624591,6.506152,3976.0,434.0,1078.0,1577.0,987.0,512.0,77.0,37.0,2157.0
2,72,"Peñuelas Municipio, Puerto Rico",21661.0,36.4,16868.0,7983.0,12379.0,57.148793,7.391164,7132.0,1361.0,2329.0,3877.0,3090.0,736.0,181.0,27.0,2972.0
3,72,"Ponce Municipio, Puerto Rico",148863.0,39.5,16561.0,10775.0,75187.0,50.507514,4.711043,53341.0,5613.0,18366.0,25211.0,17691.0,6802.0,1210.0,510.0,25042.0
4,72,"San Sebastián Municipio, Puerto Rico",38970.0,42.0,14275.0,8072.0,21472.0,55.098794,8.486015,14278.0,1525.0,2446.0,4743.0,2737.0,1335.0,829.0,456.0,8622.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,55,"Iowa County, Wisconsin",23576.0,42.5,60017.0,31717.0,2100.0,8.907363,1.895996,9799.0,1005.0,5301.0,7486.0,4180.0,2566.0,1177.0,560.0,1893.0
3216,55,"Pepin County, Wisconsin",7282.0,45.8,51470.0,27901.0,938.0,12.881077,1.757759,3005.0,224.0,1969.0,2321.0,1430.0,813.0,168.0,35.0,583.0
3217,55,"Ozaukee County, Wisconsin",87817.0,43.9,80526.0,45820.0,5087.0,5.792728,1.637496,35044.0,1978.0,26993.0,29934.0,20099.0,9122.0,1763.0,277.0,4033.0
3218,55,"Portage County, Wisconsin",70371.0,36.6,54620.0,28363.0,8757.0,12.444047,2.569240,28072.0,2469.0,19121.0,22790.0,14096.0,7623.0,1635.0,614.0,4290.0


### Create new columns to store the coordinate in the dataframe

In [52]:
# Add columns for lat, lng
coordinates_pd["Lat"] = 0
coordinates_pd["Lng"] = 0
coordinates_pd.head()

,State,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,Households with Internet,Households with Just Cell Data Plan,Households with DSL or Fiber Optic,Households with Broadband Any Type,Households with Celular Data Plan,Households with Just DSL or Fiber Optic,Households with Satellite Internet,Households with Just Satellite Internet,Households with No Internet,Lat,Lng
0,72,"Corozal Municipio, Puerto Rico",34933.0,38.1,14752.0,7887.0,19146.0,54.807775,9.435205,11026.0,2381.0,2268.0,5021.0,4088.0,706.0,361.0,161.0,5686.0,0,0
1,72,"Maunabo Municipio, Puerto Rico",11297.0,43.5,17636.0,8830.0,5945.0,52.624591,6.506152,3976.0,434.0,1078.0,1577.0,987.0,512.0,77.0,37.0,2157.0,0,0
2,72,"Peñuelas Municipio, Puerto Rico",21661.0,36.4,16868.0,7983.0,12379.0,57.148793,7.391164,7132.0,1361.0,2329.0,3877.0,3090.0,736.0,181.0,27.0,2972.0,0,0
3,72,"Ponce Municipio, Puerto Rico",148863.0,39.5,16561.0,10775.0,75187.0,50.507514,4.711043,53341.0,5613.0,18366.0,25211.0,17691.0,6802.0,1210.0,510.0,25042.0,0,0
4,72,"San Sebastián Municipio, Puerto Rico",38970.0,42.0,14275.0,8072.0,21472.0,55.098794,8.486015,14278.0,1525.0,2446.0,4743.0,2737.0,1335.0,829.0,456.0,8622.0,0,0


### Loop to consult the coordinates using the maps API

#### Warning: Looping through 3,219 rows takes about 15 minutes

In [53]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

#i = 0

# Loop through the cities_pd and run a lat/long search for each city
for index, row in coordinates_pd.iterrows():
#    if i <= 1:
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"

        county = row['Name']

        # update address key value
        params['address'] = f"{county}"

        # make request
        county_lat_lng = requests.get(base_url, params=params)
        
        # print(county_lat_lng.url)
    
        # convert to json
        county_lat_lng = county_lat_lng.json()

        # pprint.pprint(county_lat_lng)
        
        # print(county_lat_lng['results'][0]["geometry"]["location"]["lat"])
        try:
            coordinates_pd.loc[index, "Lat"] = county_lat_lng['results'][0]["geometry"]["location"]["lat"]
            coordinates_pd.loc[index, "Lng"] = county_lat_lng['results'][0]["geometry"]["location"]["lng"]
            print(f"Retrieving information for '{county}'")
        except (KeyError, IndexError):
            print("Missing field/result... skipping.")
        print("-------------------------------------------------------------------------")
#    i += 1
# Visualize to confirm lat lng appear


Retrieving information for 'Corozal Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Maunabo Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Peñuelas Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Ponce Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'San Sebastián Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Toa Alta Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Toa Baja Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Vieques Municipio, Puerto R

Retrieving information for 'Gurabo Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Salinas Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Barceloneta Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Loíza Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Yauco Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Guaynabo Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Aguadilla Municipio, Puerto Rico'
-------------------------------------------------------------------------
Retrieving information for 'Cabo Rojo Municipio, Puerto Rico

Retrieving information for 'Perry County, Alabama'
-------------------------------------------------------------------------
Retrieving information for 'Walker County, Alabama'
-------------------------------------------------------------------------
Retrieving information for 'Cleburne County, Alabama'
-------------------------------------------------------------------------
Retrieving information for 'Greene County, Alabama'
-------------------------------------------------------------------------
Retrieving information for 'Barbour County, Alabama'
-------------------------------------------------------------------------
Retrieving information for 'Bibb County, Alabama'
-------------------------------------------------------------------------
Retrieving information for 'Bullock County, Alabama'
-------------------------------------------------------------------------
Retrieving information for 'Calhoun County, Alabama'
----------------------------------------------------------------

Retrieving information for 'Pinal County, Arizona'
-------------------------------------------------------------------------
Retrieving information for 'White County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Nevada County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'St. Francis County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Pulaski County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Clay County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Cross County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Dallas County, Arkansas'
----------------------------------------------------------

Retrieving information for 'Lawrence County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Monroe County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Polk County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Conway County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Ouachita County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Newton County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Arkansas County, Arkansas'
-------------------------------------------------------------------------
Retrieving information for 'Little River County, Arkansas'
------------------------------------------------

...

Retrieving information for 'Buena Vista city, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Falls Church city, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Manassas Park city, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Petersburg city, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Poquoson city, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Sussex County, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Fairfax city, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Waynesboro city, Virginia'
-------------------------------------------

Retrieving information for 'Powhatan County, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Rockbridge County, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Shenandoah County, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Fauquier County, Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Harrison County, West Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Ohio County, West Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Ritchie County, West Virginia'
-------------------------------------------------------------------------
Retrieving information for 'Calhoun County, West Virginia'
------------------------

Retrieving information for 'Benton County, Washington'
-------------------------------------------------------------------------
Retrieving information for 'Columbia County, Washington'
-------------------------------------------------------------------------
Retrieving information for 'Clark County, Washington'
-------------------------------------------------------------------------
Retrieving information for 'Asotin County, Washington'
-------------------------------------------------------------------------
Retrieving information for 'Adams County, Washington'
-------------------------------------------------------------------------
Retrieving information for 'Garfield County, Washington'
-------------------------------------------------------------------------
Retrieving information for 'Mason County, Washington'
-------------------------------------------------------------------------
Retrieving information for 'Skagit County, Washington'
-----------------------------------------

Retrieving information for 'Lafayette County, Wisconsin'
-------------------------------------------------------------------------
Retrieving information for 'Monroe County, Wisconsin'
-------------------------------------------------------------------------
Retrieving information for 'Pierce County, Wisconsin'
-------------------------------------------------------------------------
Retrieving information for 'Wood County, Wisconsin'
-------------------------------------------------------------------------
Retrieving information for 'Ashland County, Wisconsin'
-------------------------------------------------------------------------
Retrieving information for 'Calumet County, Wisconsin'
-------------------------------------------------------------------------
Retrieving information for 'Florence County, Wisconsin'
-------------------------------------------------------------------------
Retrieving information for 'Jefferson County, Wisconsin'
------------------------------------------

Retrieving information for 'Ozaukee County, Wisconsin'
-------------------------------------------------------------------------
Retrieving information for 'Portage County, Wisconsin'
-------------------------------------------------------------------------
Retrieving information for 'Richland County, Wisconsin'
-------------------------------------------------------------------------


### Get the DataFrame with the Coordinates

In [54]:
coordinates_pd.head(50)

,State,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,Households with Internet,Households with Just Cell Data Plan,Households with DSL or Fiber Optic,Households with Broadband Any Type,Households with Celular Data Plan,Households with Just DSL or Fiber Optic,Households with Satellite Internet,Households with Just Satellite Internet,Households with No Internet,Lat,Lng
0,72,"Corozal Municipio, Puerto Rico",34933.0,38.1,14752.0,7887.0,19146.0,54.807775,9.435205,11026.0,2381.0,2268.0,5021.0,4088.0,706.0,361.0,161.0,5686.0,18.341062,-66.316836
1,72,"Maunabo Municipio, Puerto Rico",11297.0,43.5,17636.0,8830.0,5945.0,52.624591,6.506152,3976.0,434.0,1078.0,1577.0,987.0,512.0,77.0,37.0,2157.0,18.007189,-65.899329
2,72,"Peñuelas Municipio, Puerto Rico",21661.0,36.4,16868.0,7983.0,12379.0,57.148793,7.391164,7132.0,1361.0,2329.0,3877.0,3090.0,736.0,181.0,27.0,2972.0,18.063358,-66.727390
3,72,"Ponce Municipio, Puerto Rico",148863.0,39.5,16561.0,10775.0,75187.0,50.507514,4.711043,53341.0,5613.0,18366.0,25211.0,17691.0,6802.0,1210.0,510.0,25042.0,18.011077,-66.614062
4,72,"San Sebastián Municipio, Puerto Rico",38970.0,42.0,14275.0,8072.0,21472.0,55.098794,8.486015,14278.0,1525.0,2446.0,4743.0,2737.0,1335.0,829.0,456.0,8622.0,18.335476,-66.994679
5,72,"Toa Alta Municipio, Puerto Rico",74169.0,37.1,29672.0,14162.0,22070.0,29.756367,5.751729,22205.0,3316.0,8516.0,12769.0,7661.0,4296.0,1404.0,474.0,8844.0,18.388282,-66.248224
6,72,"Toa Baja Municipio, Puerto Rico",81905.0,38.9,24583.0,12240.0,29279.0,35.747512,8.015384,27802.0,5744.0,8953.0,15429.0,12312.0,2742.0,1222.0,227.0,11565.0,18.444471,-66.254329
7,72,"Vieques Municipio, Puerto Rico",8931.0,41.4,16261.0,11136.0,3260.0,36.502071,4.792296,2555.0,420.0,565.0,1124.0,825.0,235.0,156.0,47.0,1359.0,18.147462,-65.444881
8,72,"Yabucoa Municipio, Puerto Rico",35025.0,41.6,15586.0,8672.0,18297.0,52.239829,7.665953,11989.0,1386.0,2963.0,5199.0,3134.0,1604.0,1041.0,380.0,5787.0,18.050520,-65.879329
9,72,"Aguada Municipio, Puerto Rico",39470.0,41.0,16199.0,9001.0,20558.0,52.085128,6.430200,12824.0,1710.0,4179.0,6228.0,3175.0,2648.0,396.0,153.0,5800.0,18.379391,-67.188237


### Export the data

In [55]:
coordinates_pd.to_csv("output/coordinates_pd.csv", encoding="utf-8", index=False)